In [1]:
from table_creations import *
import requests
from credentials import config
from authentication import get_token
import pandas as pd
import time

In [2]:
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect() #Kan vel være et argument til funksjonen? Tenkte på dette

In [3]:
create_keyspace(session, keyspace='fish_data')
reset_table(session, keyspace='fish_space', table_name='localities')

In [4]:
create_locality_table(session)

In [5]:
from api_creations import *
df = api_weather_station_id(id = "SN65310", year="2022")
df.head()

,mean(air_temperature P1D),mean(wind_speed P1D),mean(relative_humidity P1D),week,year,referenceTime,sourceId
0,4.6,6.4,72,0,2022,2022-01-01T00:00:00.000Z,SN65310:0
0,4.0,10.2,86,0,2022,2022-01-02T00:00:00.000Z,SN65310:0
0,6.3,7.0,90,1,2022,2022-01-03T00:00:00.000Z,SN65310:0
0,3.3,6.6,77,1,2022,2022-01-04T00:00:00.000Z,SN65310:0
0,1.1,8.7,78,1,2022,2022-01-05T00:00:00.000Z,SN65310:0


In [6]:
def get_week_summary_locality(token, year, week, localityID):
  url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{localityID}/{year}/{week}"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }

  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()  

In [7]:
def get_week_summary(token, year, week):
  url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{year}/{week}"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }

  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()
import time

In [8]:
token = get_token()
year = 2022
localityNo = 30977
#Kepp these values year INT,
"""     week INT,
    localityno INT,
    datetime TEXT,
    avgadultfemalelice FLOAT,
    hasreportedlice BOOLEAN,
    avgmobilelice FLOAT,
    avgstationarylice FLOAT,
    seatemperature FLOAT, """
values_to_keep = ["week", "year", "localityNo", "avgAdultFemaleLice", "hasReportedLice", "avgMobileLice", "avgStationaryLice", "seaTemperature"]

Token request successful


In [9]:

from table_creations import *
from api_creations import *
from use_spark import *
localityNo = 30977

In [10]:
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect() #Kan vel være et argument til funksjonen? Tenkte på dette

In [11]:
token = get_token()
df_locality = pd.DataFrame()
for week in range(1, 53):

    row = {}
    locality_data = get_week_summary_locality(token, year, week, localityNo)
    for val in values_to_keep:
        row[val.lower()] = locality_data["localityWeek"][val]
    row["localityname"] = locality_data["localityName"]
    
    df_locality = pd.concat([df_locality, pd.DataFrame([row])], ignore_index=True)

Token request successful


/tmp/ipykernel_61819/3913724197.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_locality = pd.concat([df_locality, pd.DataFrame([row])], ignore_index=True)


In [12]:
create_localities_table(session, locality_id = localityNo)

In [13]:
spark = initiate_spark()

your 131072x1 screen size is bogus. expect trouble
23/11/29 13:49:43 WARN Utils: Your hostname, Ivars-PC resolves to a loopback address: 127.0.1.1; using 172.23.6.83 instead (on interface eth0)
23/11/29 13:49:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ivholmlu/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ivholmlu/.ivy2/cache
The jars for the packages stored in: /home/ivholmlu/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1f9b25ce-7b0e-4fe4-854a-ff2273fb8ee6;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0

In [14]:
df_locality.head()

,week,year,localityno,avgadultfemalelice,hasreportedlice,avgmobilelice,avgstationarylice,seatemperature,localityname
0,1,2022,30977,0.00,True,0.10,0.05,4.0,Æsøya
1,2,2022,30977,0.01,True,0.12,0.07,4.9,Æsøya
2,3,2022,30977,0.00,True,0.10,0.05,5.2,Æsøya
3,4,2022,30977,0.00,True,0.02,0.00,5.0,Æsøya
4,5,2022,30977,NaN,False,NaN,NaN,3.8,Æsøya


In [15]:
def insert_into_locality(spark, locality_id, df, keyspace="fish_data"):
    #make spark dataframe from df
    spark_df = spark.createDataFrame(df)
    spark_df.write\
        .format("org.apache.spark.sql.cassandra")\
        .mode('append')\
        .options(table=f"locality_{locality_id}", keyspace=keyspace)\
        .save()


In [16]:
insert_into_locality(spark, localityNo, df_locality)

/home/ivholmlu/.local/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


In [18]:
def get_locality_table(session, locality_id, keyspace="fish_data"):
    #Return as pandas dataframe

    query = f"SELECT * FROM {keyspace}.locality_{locality_id}"
    df = pd.DataFrame(list(session.execute(query)))
    return df

In [19]:
df_locality = get_locality_table(session, locality_id=localityNo)
df_locality.head()

,year,week,localityno,avgadultfemalelice,avgmobilelice,avgstationarylice,datetime,hasreportedlice,localityname,seatemperature
0,2022,1,30977,0.00,0.10,0.05,None,True,Æsøya,4.0
1,2022,2,30977,0.01,0.12,0.07,None,True,Æsøya,4.9
2,2022,3,30977,0.00,0.10,0.05,None,True,Æsøya,5.2
3,2022,4,30977,0.00,0.02,0.00,None,True,Æsøya,5.0
4,2022,5,30977,NaN,NaN,NaN,None,False,Æsøya,3.8
